# Прогнозирование оттока клиентов в Телекоме

## Описание проекта
Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.
Компания предоставляет два основных типа услуг:
1. Стационарная телефонная связь.
2. Интернет.
В зависимости от технических возможностей и желания клиентов у каждой из этих услуг могут быть вариации и дополнения. Кроме этого, предлагаются различные условия оплаты.
Перед нами как специалистами поставлена задача получения модели с максимальным значением AUC-ROC. Этот показатель сложен для интерпретации, поэтому необходимо обращать внимание и на другие метрики – более понятные заказчикам.


## Загрузка и описание данных
Данные о клиентах, условиях договоров и предоставляемом наборе услуг получены из разных источников. Из-за этого они распределены по четырем файлам. В дальнейшем записи будет нужно объединить, но пока загрузим и рассмотрим их по отдельности.
Но сначала загрузим все библиотеки, которые понадобятся в дальнейшем проекте.


In [1]:
import numpy as np
import pandas as pd
import phik
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_validate, train_test_split)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, OneHotEncoder,
                                StandardScaler, PolynomialFeatures)
from sklearn.svm import SVC

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

In [3]:
# отключим уведомления, которые явно будут появляться
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore', '.*One or more .*', )
warnings.filterwarnings('ignore', '.*is not supported for the liblinear .*', )


In [4]:
# Загрузим данные
contract = pd.read_csv('data/contract.csv')
internet = pd.read_csv('data/internet.csv')
personal = pd.read_csv('data/personal.csv')
phone = pd.read_csv('data/phone.csv')

In [5]:
def data_review(data, name):
    """Функция предоставляет основную информацию о DataFrame,
    переданном в качестве аргумента"""

    print(f'Размер таблицы "{name}"', data.shape)
    print(f'Количество явных дубликатов в таблице "{name}"', data.duplicated().sum())
    print(f'Образцы строк таблицы "{name}"')
    display(data.sample(10))
    print(f'Основная информация о столбцах таблицы "{name}"')
    data.info()

In [6]:
def columns_review(data, no_watch_list=[]):
    """Функция показывает частоту различных значений столбцов DataFrame, 
    относящихся к типу object. Опционально можно передать список 
    имен столбцов, которые функция будет пропускать."""

    for name in data.columns:
        if data[name].dtypes == 'object' and name not in no_watch_list:
            print(name)
            display(data[name].value_counts(normalize=True))

Данные загружены, подготовлены две вспомогательные функции. Рассмотрим каждую из имеющихся таблиц.

In [7]:
data_review(contract, 'contract');

Размер таблицы "contract" (7043, 8)
Количество явных дубликатов в таблице "contract" 0
Образцы строк таблицы "contract"


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
1156,0621-CXBKL,2015-09-01,No,Two year,No,Mailed check,18.70,1005.7
5607,2460-FPSYH,2016-08-01,2019-12-01 00:00:00,Month-to-month,Yes,Electronic check,55.80,2109.35
536,0621-HJWXJ,2014-11-01,No,Month-to-month,Yes,Bank transfer (automatic),81.55,5029.05
6540,5203-XEHAX,2017-04-01,No,One year,No,Electronic check,64.35,2053.05
1617,4939-KYYPY,2017-11-01,No,Month-to-month,No,Electronic check,59.45,1611.65
6598,6169-PGNCD,2015-05-01,No,Two year,Yes,Credit card (automatic),74.30,4166.35
4868,2568-OIADY,2016-12-01,2020-01-01 00:00:00,Month-to-month,Yes,Electronic check,99.50,3762
5606,6586-PSJOX,2018-11-01,No,One year,No,Credit card (automatic),55.20,864.55
3341,1125-SNVCK,2016-01-01,No,Month-to-month,No,Mailed check,43.80,2106.05
2434,1465-WCZVT,2019-11-01,No,Month-to-month,No,Mailed check,19.65,60.65


Основная информация о столбцах таблицы "contract"
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


В таблице contract 7043 строки и 8 столбцов. Явные дубликаты и пропуски, на первый взгляд, отсутствуют.
Столбец «customerID» содержит индивидуальные идентификаторы для клиентов. Скорее всего, они уникальны для каждого контракта. Позднее мы это проверим.
Следующими идут два столбца, похожие на даты. Это день заключения и расторжения договора. Первый из них, скорее всего, удастся легко преобразовать в дату. Второй – едва ли. Там большое количество значений «No», означающих то, что договор еще действует. Таким образом, после преобразования на основе столбца «EndDate» будет создан целевой признак.
Далее расположена группа категориальных признаков, характеризующих условия платежей.
Последние два столбца – численные. Они показывают размер ежемесячной и суммарной платы по каждому договору. Обратим внимание на то, что столбец «TotalCharges» имеет неожиданный тип. Далее мы также попробуем выяснить причины этого.


In [8]:
print(contract['customerID'].duplicated().sum())

0


Повторений в «customerID» нет. Попробуем проверить, мешает ли что-то преобразовать два других столбца в ожидаемый тип.

In [9]:
display(contract['BeginDate'].sort_values())
display(contract['TotalCharges'].sort_values())

4513    2013-10-01
4610    2013-10-01
3439    2013-10-01
975     2013-11-01
3040    2013-11-01
           ...    
3331    2020-02-01
6670    2020-02-01
936     2020-02-01
3826    2020-02-01
6754    2020-02-01
Name: BeginDate, Length: 7043, dtype: object

936           
3826          
4380          
753           
5218          
         ...  
6646    997.75
5598     998.1
3686    999.45
3353     999.8
2845     999.9
Name: TotalCharges, Length: 7043, dtype: object

Даты выглядят нормально. Сортировка дала ожидаемый результат, формат везде совпадает. Похоже, что проблем со сменой типа не будет. А вот общая сумма платежей не везде выражена числами. Здесь есть странные значения в виде пробелов. Посмотрим, что это за записи.

In [10]:
display(contract[contract['TotalCharges'] == ' '])
display(contract[contract['BeginDate'] == '2020-02-01'])

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
488,4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,
753,3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,
936,5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,
1082,4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,
1340,1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,
3331,7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,
3826,3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,
4380,2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,
5218,2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,
6670,4075-WKNIU,2020-02-01,No,Two year,No,Mailed check,73.35,


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
488,4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,
753,3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,
936,5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,
1082,4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,
1340,1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,
3331,7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,
3826,3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,
4380,2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,
5218,2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,
6670,4075-WKNIU,2020-02-01,No,Two year,No,Mailed check,73.35,


Фактически это пропуски. Далее мы решим, что с ними делать. Возникли они у всех без исключения контрактов, заключенных 01.02.2020. Это подтверждается тем, что две таблицы выше полностью совпадают. Кстати, плата за месяц здесь определена. Возможно, это число прописано в договоре, а общая сумма формируется на основе сделанных платежей.
Более подробно рассмотрим имеющиеся в таблице «contract» данные.


In [11]:
columns_review(contract, ['BeginDate', 'customerID', 'TotalCharges'])

EndDate


No                     0.734630
2019-11-01 00:00:00    0.068863
2019-12-01 00:00:00    0.066165
2020-01-01 00:00:00    0.065313
2019-10-01 00:00:00    0.065029
Name: EndDate, dtype: float64

Type


Month-to-month    0.550192
Two year          0.240664
One year          0.209144
Name: Type, dtype: float64

PaperlessBilling


Yes    0.592219
No     0.407781
Name: PaperlessBilling, dtype: float64

PaymentMethod


Electronic check             0.335794
Mailed check                 0.228880
Bank transfer (automatic)    0.219225
Credit card (automatic)      0.216101
Name: PaymentMethod, dtype: float64

Обращает на себя внимание колонка «EndDate». Во-первых, значения «No», они же будущие нули в целевом признаке, составляют 73 % значений. Таким образом, наблюдается дисбаланс. Во-вторых, есть всего четыре даты, в которые разрывались контракты. Удастся ли извлечь пользу из этих «юрьевых дней» неизвестно, но факт любопытный.
В остальных столбцах бинарные и небинарные качественные признаки. Перейдем к числовым значениям.


In [12]:
pd.concat(
    [contract['MonthlyCharges'].value_counts(normalize=True, bins=20, sort=False),
    contract['MonthlyCharges'].value_counts(normalize=True, bins=20, sort=False).cumsum()],
    axis=1)

,MonthlyCharges,MonthlyCharges
"(18.148999999999997, 23.275]",0.168536,0.168536
"(23.275, 28.3]",0.059492,0.228028
"(28.3, 33.325]",0.012069,0.240097
"(33.325, 38.35]",0.015050,0.255147
"(38.35, 43.375]",0.014482,0.269629
"(43.375, 48.4]",0.037342,0.306971
"(48.4, 53.425]",0.043873,0.350845
"(53.425, 58.45]",0.046997,0.397842
"(58.45, 63.475]",0.035212,0.433054
"(63.475, 68.5]",0.032089,0.465143


В платежах за месяц мы видим несколько пиков. Первый и самый крупный приходится на минимальные значения показателя. Видимо, заметная часть клиентов пользуется самыми дешевыми тарифами без дополнительных услуг. Еще два пика – это 55 и 80 долларов. Возможно это какие-то популярные пакеты услуг.



In [13]:
contract['MonthlyCharges'].value_counts().head(20)

20.05    61
19.85    45
19.95    44
19.90    44
20.00    43
19.70    43
19.65    43
19.55    40
20.15    40
19.75    39
20.25    39
20.35    38
19.80    38
20.10    37
19.60    37
20.20    35
19.50    32
20.45    31
19.40    31
20.40    30
Name: MonthlyCharges, dtype: int64

Мы видим, что с одной стороны, топ 20 наиболее распространенных месячных платежей находятся около 20 долларов, с другой – они все разные. На основе этих данных не похоже, что у всех один и тот же тариф. Трудно поверить, что у провайдера динамическое ценообразование, но пока никаких разумных объяснений этому факту предложить не могу.

In [14]:
pd.concat(
    [contract.loc[contract['TotalCharges'] != ' ', 'TotalCharges'].astype('float').value_counts(normalize=True, bins=20, sort=False),
    contract.loc[contract['TotalCharges'] != ' ', 'TotalCharges'].astype('float').value_counts(normalize=True, bins=20, sort=False).cumsum()],
    axis=1)

,TotalCharges,TotalCharges
"(10.133000000000001, 452.1]",0.267491,0.267491
"(452.1, 885.4]",0.117747,0.385239
"(885.4, 1318.7]",0.097696,0.482935
"(1318.7, 1752.0]",0.076934,0.559869
"(1752.0, 2185.3]",0.052332,0.612201
"(2185.3, 2618.6]",0.043089,0.655290
"(2618.6, 3051.9]",0.037400,0.692691
"(3051.9, 3485.2]",0.034841,0.727531
"(3485.2, 3918.5]",0.031712,0.759243
"(3918.5, 4351.8]",0.032850,0.792093


В показанном разрезе мы видим сначала резкий, а потом пологий спуск суммарного размера платежей по контрактам. Это может быть связано со временем взаимодействия с клиентом – чем дольше – тем больше накоплено. Но и многолетних клиентов не так много, как относительных новичков. Так как у нас большой массив наблюдений попал в одну корзину, рассмотрим ближе участок до тысячи долларов.

In [15]:
contract.loc[contract['TotalCharges'] != ' ', 'TotalCharges'].astype('float').value_counts(normalize=True, bins=100, sort=False).head(12)

(10.133000000000001, 105.46]    0.117890
(105.46, 192.12]                0.046075
(192.12, 278.78]                0.040956
(278.78, 365.44]                0.033703
(365.44, 452.1]                 0.028868
(452.1, 538.76]                 0.029010
(538.76, 625.42]                0.023606
(625.42, 712.08]                0.022042
(712.08, 798.74]                0.021189
(798.74, 885.4]                 0.021900
(885.4, 972.06]                 0.020762
(972.06, 1058.72]               0.019625
Name: TotalCharges, dtype: float64

На этом участке общая тенденция подтверждается – идет постепенное снижение накопленных платежей. Далее перейдем к другим таблицам.

In [16]:
data_review(internet, 'internet')

Размер таблицы "internet" (5517, 8)
Количество явных дубликатов в таблице "internet" 0
Образцы строк таблицы "internet"


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2915,9057-SIHCH,Fiber optic,No,No,No,No,Yes,Yes
2962,4958-XCBDQ,Fiber optic,No,No,Yes,No,Yes,Yes
3591,1741-WTPON,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes
5268,1960-UOTYM,DSL,Yes,Yes,Yes,No,Yes,Yes
3507,3714-JTVOV,Fiber optic,Yes,No,No,No,No,No
4757,3565-UNOCC,Fiber optic,No,No,Yes,No,Yes,Yes
1619,9734-UYXQI,DSL,No,No,No,No,No,No
5354,8738-JOKAR,DSL,Yes,Yes,No,Yes,No,Yes
3953,5673-TIYIB,DSL,No,Yes,Yes,Yes,No,No
4145,2082-OJVTK,Fiber optic,No,No,Yes,No,No,Yes


Основная информация о столбцах таблицы "internet"
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


В таблице «internet» представлены все те же ID пользователей, а также категориальные переменные, описывающие услуги для каждого из них. Проверим, есть ли повторяющиеся ID, а также, насколько популярны различные услуги.

In [17]:
print('Количество повторений customerID в таблице', internet['customerID'].duplicated().sum())
columns_review(internet, ['customerID', ])

Количество повторений customerID в таблице 0
InternetService


Fiber optic    0.561175
DSL            0.438825
Name: InternetService, dtype: float64

OnlineSecurity


No     0.63404
Yes    0.36596
Name: OnlineSecurity, dtype: float64

OnlineBackup


...

No     0.629509
Yes    0.370491
Name: TechSupport, dtype: float64

StreamingTV


No     0.509335
Yes    0.490665
Name: StreamingTV, dtype: float64

StreamingMovies


No     0.504803
Yes    0.495197
Name: StreamingMovies, dtype: float64

ID пользователей в таблице не повторяются: один контракт – один комплект услуг. Это упростит объединение таблиц. Все остальные столбцы – это бинарные категориальные переменные, причем распределенные без гигантских перекосов.
Теперь посмотрим на личные данные клиентов.


In [18]:
data_review(personal, 'personal')

Размер таблицы "personal" (7043, 5)
Количество явных дубликатов в таблице "personal" 0
Образцы строк таблицы "personal"


,customerID,gender,SeniorCitizen,Partner,Dependents
802,8859-AXJZP,Male,0,Yes,Yes
1068,5536-RTPWK,Male,0,Yes,No
1332,4656-CAURT,Male,0,No,No
4767,1064-FBXNK,Male,0,Yes,Yes
4981,8008-HAWED,Male,0,No,No
542,2866-IKBTM,Female,0,No,No
5091,2606-RMDHZ,Male,0,No,No
1565,6968-GMKPR,Female,0,No,No
936,5709-LVOEQ,Female,0,Yes,Yes
4719,0362-ZBZWJ,Male,0,No,No


Основная информация о столбцах таблицы "personal"
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


Количество записей с персональными данными соответствует размеру таблицы «contract». Как и раньше, мы проверим количество уникальных значений в «customerID», но сомнений тут мало. Почему-то столбец «SeniorCitizen» представлен числовыми значениями. Но выглядит он как обычный категориальный признак после onehotencoder.

In [19]:
print('Количество повторений customerID в таблице', personal['customerID'].duplicated().sum())
columns_review(personal, ['customerID', ])
display(personal['SeniorCitizen'].value_counts(normalize=True))

Количество повторений customerID в таблице 0
gender


Male      0.504756
Female    0.495244
Name: gender, dtype: float64

Partner


No     0.516967
Yes    0.483033
Name: Partner, dtype: float64

Dependents


No     0.700412
Yes    0.299588
Name: Dependents, dtype: float64

0    0.837853
1    0.162147
Name: SeniorCitizen, dtype: float64

Здесь разница в частоте некоторых категорий значительная. Зато все они бинарные, то есть существенного роста числа признаков не ожидается.

In [20]:
data_review(phone, 'phone')

Размер таблицы "phone" (6361, 2)
Количество явных дубликатов в таблице "phone" 0
Образцы строк таблицы "phone"


,customerID,MultipleLines
4018,5449-FIBXJ,Yes
2978,6504-VBLFL,No
2936,5402-HTOTQ,Yes
5895,1794-SWWKL,Yes
5409,3842-QTGDL,No
2036,0788-DXBFY,No
3614,8515-OCTJS,Yes
3749,3192-LNKRK,No
2593,9371-BITHB,No
3370,9537-JALFH,No


Основная информация о столбцах таблицы "phone"
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [21]:
print('Количество повторений customerID в таблице', phone['customerID'].duplicated().sum())
columns_review(phone, ['customerID', ])

Количество повторений customerID в таблице 0
MultipleLines


No     0.532935
Yes    0.467065
Name: MultipleLines, dtype: float64

Самая маленькая из таблиц. Здесь также нет дубликатов, повторений в ID клиентов, а единственный признак является бинарным.
Обратим внимание, что в таблицах с описанием предоставляемых услуг по отдельности строк меньше, чем в таблице с контрактами. А вот суммарно в них строк больше. Значит, некоторые клиенты получают только одну услугу – телефонная связь или интернет, а другие – обе.


In [22]:
len(pd.concat([internet['customerID'], phone['customerID']]).unique())

7043

Контрактов без услуг у нас нет. На этом заканчиваем исследование данных
*Основные выводы:*
- отсутствуют дубликаты и почти отсутствуют пропуски. Подготовка данных будет простой.
- Есть столбец, который позволит легко объединить все таблицы.
- Данные почти полностью состоят из категориальных переменных. Исключением являются Id клиентов и две колонки с платежами.
- Данные не содержат явного целевого показателя. Его необходимо получить из одного из столбцов.
На основе проведенного исследования можно сформировать приблизительный план дальнейших действий.


## План исследования
1. Подготовка данных: объединение таблиц, заполнение пропусков, смена типов, создание столбца с целевым показателем. Здесь же, видимо, применение кодирования категориальных признаков, но с сохранением оригинальной таблицы, так как, возможно, будут применяться альтернативные OHE методы.
2. Разведочный анализ данных: сравнение характеристик ушедших и оставшихся пользователей, оценка корреляции признаков. Возможно здесь также будут созданы дополнительные признаки.
3. Построение первых моделей и оценка важности признаков: разделение на тестовую и обучающую выборку, обучение моделей без подбора гиперпараметров для выбора наиболее перспективных из них, оценка важности признаков на основе двух моделей.
4. Подбор гиперпараметров для 2-4 наиболее перспективных моделей, проверка лучшей на тестовой выборке.


## Подготовка данных
Начнем подготовку данных с создания столбца с целевым показателем и объединения таблиц.


In [23]:
data = contract.merge(
    personal, on='customerID', how='left').merge(
    internet, on='customerID', how='left').merge(
    phone, on='customerID', how='left')

In [24]:
data['exited'] = data['EndDate'].apply(lambda x: 0 if x == 'No' else 1)

In [25]:
print('Размер объединенной таблицы', data.shape)
print('Количество пропущенных значений', data.isna().sum().sum())

Размер объединенной таблицы (7043, 21)
Количество пропущенных значений 11364


Мы получили ожидаемый результат. Количество строк равно количеству в базовой таблице «Contract», Количество столбцов равно сумме в четырех таблицах минус повторяющиеся плюс целевой показатель. Пропуски появились в тех случаях, когда клиенту не оказываются услуги по одному из двух направлений. Очевидно, что подобные пропуски необходимо заменить значением «No».

In [26]:
data['MultipleLines'] = data['MultipleLines'].fillna('Nothing')
data['InternetService'] = data['InternetService'].fillna('Nothing')
data = data.fillna('No')
print('Количество пропущенных значений', data.isna().sum().sum())

Количество пропущенных значений 0


Теперь вспомним, что в столбце «TotalCharges» есть пробелы. Скорее всего, по этим контрактам еще не обработан ни один платеж, то есть пробелы можно заменить нулями.

In [27]:
data['TotalCharges'] = data['TotalCharges'].replace(' ', '0')

В дальнейшем мы планируем использовать «EndDate» » для определения времени, в течение которого действует или действовал контракт. Здесь встречается значение «No». Его заменим датой выгрузки данных, то есть «2020-02-01 00:00:00»

In [28]:
data['EndDate'] = data['EndDate'].replace('No', '2020-02-01 00:00:00')

Теперь мы можем преобразовать столбцы с датами и числами в соответствующие типы

In [29]:
data['BeginDate'] = pd.to_datetime(data['BeginDate'])
data['EndDate'] = pd.to_datetime(data['EndDate'])
data['TotalCharges'] = data['TotalCharges'].astype('float')

In [30]:
data['BeginDate'] = pd.to_datetime(data['BeginDate'])
data['EndDate'] = pd.to_datetime(data['EndDate'])
data['TotalCharges'] = data['TotalCharges'].astype('float')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   datetime64[ns]
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   float64       
 8   gender            7043 non-null   object        
 9   SeniorCitizen     7043 non-null   int64         
 10  Partner           7043 non-null   object        
 11  Dependents        7043 non-null   object        
 12  InternetService   7043 non-null   object        
 13  OnlineSecurity    7043 non-null   object        
 14  OnlineBackup      7043 n

*Выводы*
На этом подготовка данных завершается. Нам удалось выполнить поставленные в этой части исследования задачи, а именно:
- объединить таблицы;
- выделить целевой показатель;
- заполнить пропуски;
- изменить типы данных на более подходящие.
Здесь все это проводилось постепенно, с пояснениями. В дальнейшем, все преобразования могут быть помещены в отдельную функцию, чтобы при поступлении новых данных вся обработка проводилась в одну-две строки. Пока эту ее создание отложим, так как в нее можно включить и некоторые другие действия, которые будут определены на следующем этапе проекта.


## Разведочный анализ данных
Мы уже почти получили данные, пригодные для обучения моделей. Однако, прежде чем перейти к этому увлекательному процессу, попробуем самостоятельно извлечь из них какие-либо закономерности. Возможно такой анализ подтолкнет нас к более эффективным решениям, поможет найти полезные для бизнеса факты. На данном этапе сравним, чем отличаются ушедшие и оставшиеся клиенты, есть ли существенная взаимосвязь между различными факторами.
Хотелось бы сравнение имеющихся признаков дополнить еще несколькими штрихами. Например, как влияет продолжительность взаимоотношений между компанией и клиентом на вероятность расторжения договора, какое количество услуг является залогом лояльности, что надежнее – интернет, телефон или обе эти услуги одновременно. Создадим такие признаки.


In [31]:
def internet_or_phone(row):
    if row['InternetService'] == 'Nothing': return 'phone'
    elif row['MultipleLines'] == 'Nothing': return 'internet'
    else: return 'both'
data['service'] = data.apply(internet_or_phone, axis=1)

In [32]:
def count_services(row):
    c = 0
    if row['InternetService'] != 'Nothing': c += 1
    if row['OnlineSecurity'] == 'Yes': c += 1
    if row['OnlineBackup'] == 'Yes': c += 1
    if row['DeviceProtection'] == 'Yes': c += 1
    if row['TechSupport'] == 'Yes': c += 1
    if row['StreamingTV'] == 'Yes': c += 1
    if row['StreamingMovies'] == 'Yes': c += 1
    if row['MultipleLines'] != 'Nothing': c += 1
    return c
data['number_service'] = data.apply(count_services, axis=1)

In [33]:
data['long'] = (data['EndDate'] - data['BeginDate']).dt.days

Теперь создадим таблицу, в которой различные признаки будут показаны в разрезе всей выборки, а также по отдельности для выбывших и оставшихся клиентов. Скорее всего, большая разница между тремя этими показателями будет свидетельствовать о высокой значимости признака.

In [34]:
pd.get_dummies(data.drop(['customerID', 'BeginDate', 'EndDate'], axis=1)).groupby('exited').mean().T.join(
    pd.Series(pd.get_dummies(data.drop(['customerID', 'BeginDate', 'EndDate'], axis=1)).mean(), name='share').T)

,0,1,share
MonthlyCharges,61.265124,74.441332,64.761692
TotalCharges,2549.911442,1531.796094,2279.734304
SeniorCitizen,0.128721,0.254682,0.162147
number_service,3.763239,3.616907,3.724407
long,1144.447236,547.352060,985.996166
Type_Month-to-month,0.429068,0.885500,0.550192
Type_One year,0.252609,0.088818,0.209144
Type_Two year,0.318322,0.025682,0.240664
PaperlessBilling_No,0.464438,0.250936,0.407781
PaperlessBilling_Yes,0.535562,0.749064,0.592219


Получилась большая таблица. Выберем из нее несколько интересных для бизнеса фактов.
- Ежемесячные платежи ушедших клиентов в среднем на 13 долларов больше, чем у оставшихся. Зато общие платежи оставшихся значительно выше. Возможно, со временем шансы на уход клиентов падают. На это указывает и разница в продолжительности действия контракта.
- Среди пенсионеров доля отказавшихся от услуг выше, чем среди более молодых клиентов. Возможно, к обычным причинам отказа от услуг у них добавляется риск смерти.
- Количество подключенных услуг не влияет на вероятность ухода клиента.
- Почти 90 % отказавшихся от услуг пользователей обслуживались по договорам с ежемесячной платой. Как мы видим, по таким контрактам обслуживается 55 % клиентов, то есть наблюдается более низкая лояльность этой группы.
- Получатели чеков на email отказываются от услуг в три раза чаще, но и в целом их больше.
- Почему-то клиенты, использующие электронные чеки, уходят чаще всех. Возможно, что-то с этим методам оплаты не так.
- Пол не влияет на вероятность расторжения договора.
- клиенты с супругами и детьми несколько реже расстаются с провайдером, чем их свободные от подобных обязательств коллеги.
- Похоже, что клиенты, получающие интернет с помощью оптико-волоконного кабеля не очень довольны качеством услуг. Их доля среди ушедших выше, чем их доля во всей выборке. А вот клиенты, которым интернет не предоставляют, уходят редко. Видимо, с телефонной связью все в порядке.
- Среди других услуг можно выделить значительное положительное влияние на риски ухода предоставления технической поддержки и услуг онлайн-защиты.
- Чаще всего уходят пользователи, получающие как услуги интернета, так и телефонной связи.
Теперь посмотрим пристальнее на количественные признаки.


In [35]:
for name in ['MonthlyCharges', 'TotalCharges', 'number_service', 'long']:
    print(name)
    display(data.groupby('exited')[name].describe())

MonthlyCharges


,count,mean,std,min,25%,50%,75%,max
exited,,,,,,,,
0,5174.0,61.265124,31.092648,18.25,25.10,64.425,88.4,118.75
1,1869.0,74.441332,24.666053,18.85,56.15,79.650,94.2,118.35


TotalCharges


,count,mean,std,min,25%,50%,75%,max
exited,,,,,,,,
0,5174.0,2549.911442,2329.954215,0.00,572.9,1679.525,4262.85,8672.45
1,1869.0,1531.796094,1890.822994,18.85,134.5,703.550,2331.30,8684.80


number_service


,count,mean,std,min,25%,50%,75%,max
exited,,,,,,,,
0,5174.0,3.763239,2.250114,1.0,1.0,4.0,6.0,8.0
1,1869.0,3.616907,1.609944,1.0,2.0,3.0,5.0,8.0


long


,count,mean,std,min,25%,50%,75%,max
exited,,,,,,,,
0,5174.0,1144.447236,733.897937,0.0,457.0,1157.0,1857.0,2191.0
1,1869.0,547.352060,594.389607,30.0,61.0,304.0,883.0,2191.0


Тенденции подтверждаются. Обратим внимание на то, как медианное значение «long» отличается в двух группах. Есть надежда, что этот показатель сильно повысит качество модели. 
После рассмотрения всех этих соотношений, проверим, есть ли среди признаков случаи высокой взаимосвязи. Это важно для некоторых моделей, а также может подтолкнуть к дополнительным выводам.


In [36]:
corr_matrix = data.drop('customerID', axis=1).phik_matrix()
for i in range(corr_matrix .shape[0]):
    corr_matrix.iloc[i, i] = 0
corr_matrix .stack().sort_values(ascending=False).head(30)

interval columns not set, guessing: ['MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'exited', 'number_service', 'long']


EndDate           exited              1.000000
exited            EndDate             1.000000
long              BeginDate           0.995271
BeginDate         long                0.995271
service           InternetService     0.964441
InternetService   service             0.964441
MultipleLines     service             0.952658
service           MultipleLines       0.952658
InternetService   MonthlyCharges      0.919002
MonthlyCharges    InternetService     0.919002
                  service             0.889359
service           MonthlyCharges      0.889359
number_service    StreamingMovies     0.871554
StreamingMovies   number_service      0.871554
number_service    StreamingTV         0.869423
StreamingTV       number_service      0.869423
DeviceProtection  number_service      0.864647
number_service    DeviceProtection    0.864647
long              TotalCharges        0.842146
TotalCharges      long                0.842146
StreamingTV       MonthlyCharges      0.835340
MonthlyCharge

Здесь обращает на себя внимание несколько фактов
- максимальная корреляция даты завершения договора и целевого признака. Логично, ведь один и создан на основе другого.
- Целый ряд параметров связаны с датой заключения договора и датой выгрузки данных. Между ними, продолжительностью действия договора и суммарными выплатами значительная связь.
- Высокая корреляция между сконструированными нами «service» и «number_service» с одной стороны и признаками, лежащими в их основе, с другой.
- Удивительно высокая корреляция между «InternetService» и «MonthlyCharges». Похоже, услуги телефонии стоят совсем недорого, по сравнению с интернетом.
На самом деле, использованный здесь коэффициент интерпретируется значительно сложнее, чем привычная корреляция Пирсона. Без дополнительных исследований он скорее является подсказкой, чем указанием к действию. Чтобы немного прояснить картину, посмотрим и обычную корреляцию там, где это возможно.


In [37]:
data.corr()

C:\Users\User\AppData\Local\Temp\ipykernel_9088\2627137660.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,MonthlyCharges,TotalCharges,SeniorCitizen,exited,number_service,long
MonthlyCharges,1.000000,0.651174,0.220173,0.193356,0.822187,0.247754
TotalCharges,0.651174,1.000000,0.103006,-0.198324,0.744813,0.826109
SeniorCitizen,0.220173,0.103006,1.000000,0.150889,0.096434,0.016514
exited,0.193356,-0.198324,0.150889,1.000000,-0.030765,-0.352673
number_service,0.822187,0.744813,0.096434,-0.030765,1.000000,0.443595
long,0.247754,0.826109,0.016514,-0.352673,0.443595,1.000000


Здесь все логично. Чем больше услуг, тем выше ежемесячная плата. Чем дольше действует договор, тем больше накопленная сумма платежей.
*Выводы*
На этом завершаем разведочный анализ данных. Мы обнаружили несколько отличий оставшихся пользователей, от тех, которые расторгли договор. Наиболее значимыми являются продолжительность сотрудничества на момент принятия решения, тип договора, получение электронных квитанций, способ оплаты и некоторые другие. Также мы определили несколько потенциально опасных для моделей столбцов, имеющих высокую связь между собой. От некоторых из них, скорее всего, придется избавиться.



## Построение первых моделей
Мы подошли к этапу построения первых моделей. Прежде всего, для их применения необходимо подготовить данные. В значительной степени это сделано на предыдущих этапах. При этом нужно, во-первых, стремиться к максимальной автоматизации всех процессов, во-вторых, к отдельной обработке обучающей и тестовой выборки. Создадим функцию, которая будет повторять все уже проделанные манипуляции: объединение данных, заполнение пропусков и т.д.


In [38]:
def separate_feature_and_target(data_list):
    '''Функция получает список таблиц, объединяет их,
    возвращает набор признаков и целевых показателей.
    Важно первой передать таблицу "contract".'''
    x = data_list[0]
    for d in data_list[1:]:
        x = x.merge(d, on='customerID', how='left')
    y  = x['EndDate'].apply(lambda x: 0 if x == 'No' else 1)
    x = x.drop('customerID', axis=1)
    return x, y

In [39]:
def data_preparation(data, drop_column=None):
    '''Функция заполняет пропуски и создает новые признаки'''
    
    data['MultipleLines'] = data['MultipleLines'].fillna('Nothing')
    data['InternetService'] = data['InternetService'].fillna('Nothing')
    data = data.fillna('No')

    data['TotalCharges'] = data['TotalCharges'].replace(' ', '0')

    data['EndDate'] = data['EndDate'].replace('No', '2020-02-01 00:00:00')

    data['BeginDate'] = pd.to_datetime(data['BeginDate'])
    data['EndDate'] = pd.to_datetime(data['EndDate'])
    data['TotalCharges'] = data['TotalCharges'].astype('float')

    data['service'] = data.apply(internet_or_phone, axis=1)
    data['number_service'] = data.apply(count_services, axis=1)
    data['long'] = (data['EndDate'] - data['BeginDate']).dt.days
    data = data.drop(['BeginDate', 'EndDate'], axis=1)
    if drop_column: data = data.drop(drop_column, axis=1)
    return data

Теперь подготовим несколько преобразователей для будущих конвейеров.

In [40]:
preparation = FunctionTransformer(data_preparation, kw_args={'drop_column': None})
ct_ohe = ColumnTransformer(
    [('ohe', OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'), 
      make_column_selector(dtype_include=object)),
    ('numbers', StandardScaler(), 
     make_column_selector(dtype_include=np.number))],
    remainder='passthrough')

Теперь почти все готово к построению первых моделей. Осталось отделить тестовую выборку

In [41]:
x, y = separate_feature_and_target([contract, personal, internet, phone])
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.25, shuffle=True, random_state=130323)

На первых моделях мы даже не посмотрим на качество. Попробуем разобраться, какие признаки они считают важными. Для этого слегка преобразуем обучающую выборку и передадим ее случайному лесу и линейной регрессии. Во втором случае мы будем применять регуляризацию, чтобы уменьшить коэффициенты.

In [42]:
new_x_train = data_preparation(x_train)
new_x_train = pd.get_dummies(new_x_train, drop_first=True)
scaler = StandardScaler()
new_x_train = pd.DataFrame(scaler.fit_transform(new_x_train),
                           columns=new_x_train.columns)

In [43]:
rf = RandomForestClassifier(random_state=130323)
rf.fit(new_x_train, y_train)
pd.Series(rf.feature_importances_,
    index=rf.feature_names_in_).sort_values(ascending=False)

long                                     0.220310
TotalCharges                             0.173443
MonthlyCharges                           0.148145
InternetService_Fiber optic              0.047300
number_service                           0.043078
Type_Two year                            0.033870
PaymentMethod_Electronic check           0.033420
gender_Male                              0.025541
PaperlessBilling_Yes                     0.023821
Type_One year                            0.020393
Partner_Yes                              0.020342
SeniorCitizen                            0.020278
Dependents_Yes                           0.019396
OnlineSecurity_Yes                       0.019208
TechSupport_Yes                          0.018751
MultipleLines_Yes                        0.017940
OnlineBackup_Yes                         0.017794
DeviceProtection_Yes                     0.016109
StreamingMovies_Yes                      0.014552
StreamingTV_Yes                          0.013510


In [44]:
lr = LogisticRegression(penalty='l1', solver='liblinear')
lr.fit(new_x_train, y_train)
pd.Series(np.reshape(lr.coef_, 26),
    index=lr.feature_names_in_).abs().sort_values(ascending=False)

long                                     1.519357
TotalCharges                             0.792579
Type_Two year                            0.614221
InternetService_Fiber optic              0.358954
service_phone                            0.280319
Type_One year                            0.244580
OnlineSecurity_Yes                       0.172707
PaperlessBilling_Yes                     0.166418
TechSupport_Yes                          0.146252
PaymentMethod_Electronic check           0.124695
service_internet                         0.119306
MultipleLines_Yes                        0.109903
StreamingTV_Yes                          0.094997
StreamingMovies_Yes                      0.087747
OnlineBackup_Yes                         0.084097
Dependents_Yes                           0.076830
SeniorCitizen                            0.073739
PaymentMethod_Mailed check               0.053212
MultipleLines_Nothing                    0.047478
PaymentMethod_Credit card (automatic)    0.029393


В основном две модели сходятся в своих выводах. Наибольшей значимостью обладают продолжительность взаимоотношений, суммарные платы и ежемесячный счет. Напомним, что между тремя этими показателями ранее была обнаружена корреляция. Выше не показано, но экспериментальным путем было обнаружено, что в случае исключения одного или двух признаков из этой тройки, их вес почти полностью переносится на оставшийся. При этом качество модели падает на 1-2 %.
С точки зрения интерпретации их связь мешает мало, ведь они слишком сильно выделяются на фоне других. А две сотых влияния на качество не помешают. Оставим все три.
Среди отличий обратим внимание на то, что случайный лес посчитал относительно важными пол клиента и общее количество услуг, тогда как логистическая регрессия их почти не заметила.
Теперь перейдем к оценке моделей без подбора гиперпараметров. Составим словарь, включающий модели, а затем для каждой модели создадим конвейер. Также на этом этапе начнем применять стратифицированную кросс-валидацию.


In [45]:
models = {
    'random_forest': RandomForestClassifier(random_state=130323),
    'ExtraTreesClassifier': ExtraTreesClassifier(random_state=130323),
    'DummyClassifier': DummyClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'RidgeClassifier': RidgeClassifier(),
    'LogisticRegression': LogisticRegression(),
    'LGBMClassifier': LGBMClassifier(),
    'SVC': SVC(kernel='poly', degree=7)}

In [46]:
folds = StratifiedKFold(shuffle=True, random_state=130323)

In [47]:
result = {}
for key in models:
    pipe = Pipeline(
        [('preparation', preparation),
        ('ct', ct_ohe),
        (key, models[key])])
    cross_dict = cross_validate(pipe, x_train, y_train, scoring='roc_auc', cv=folds, return_train_score=True)
    result[key] = [cross_dict['test_score'].mean(), cross_dict['train_score'].mean()]

pd.DataFrame(result, index=['valid', 'train']).T.sort_values(by='valid', ascending=False)

,valid,train
LGBMClassifier,0.887078,0.983759
LogisticRegression,0.840302,0.845142
random_forest,0.837934,0.999994
RidgeClassifier,0.831016,0.835334
ExtraTreesClassifier,0.804361,0.999999
KNeighborsClassifier,0.774999,0.893899
SVC,0.761127,0.943571
DummyClassifier,0.500000,0.500000


*Выводы*
На данном этапе исследования были подготовлены трансформаторы для будущих конвейеров. Кроме этого на примере случайного леса и логистической регрессии мы определили наиболее важные признаки. Наибольшим значением с большим отрывом от остальных обладают продолжительность отношений с клиентами, суммарный размер выплат и ежемесячные платежи. Даже значительная корреляция не подавляет значимость какого-либо из них. Среди других признаков модели выделяют, например, технологию Интернет-соединения, а также тип договора.
Среди моделей, качество которых мы проверили, выделим LGBM, случайный лес и логистическую регрессию. Они показали хорошие результаты. При этом первые две явно переобучены. Возможно, удастся подобрать гиперпараметры, которые уменьшат эту проблему. Логистическая регрессия никаких признаков переобучения не демонстрирует. Наоборот, можно попробовать усложнить ее, чтобы научить выявлять дополнительные закономерности. Подбором гиперпараметров займемся на следующем этапе.


## Настройка моделей
Ранее мы использовали одинаковые конвейеры для всех моделей. Здесь попробуем, при необходимости, создать для каждой из них отдельные. Затем подберем оптимальные параметры. Начнем, пожалуй, с модели LGBM.


In [48]:
LGBM_pipe = Pipeline(
    [('preparation', preparation),
    ('ct', ct_ohe),
    ('lgbm', LGBMClassifier(random_state=130323))])

In [49]:
param = {
    'lgbm__boosting_type': ['gbdt', 'dart'],
                            'lgbm__max_depth': [1, 2, 3],
'lgbm__n_estimators': [50, 100, 200]}
LGBM_grid = GridSearchCV(LGBM_pipe, param, scoring='roc_auc', cv=folds, return_train_score=True)
LGBM_grid.fit(x_train, y_train)
pd.DataFrame(LGBM_grid.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lgbm__boosting_type,param_lgbm__max_depth,param_lgbm__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,0.992315,0.060210,0.246465,0.011983,gbdt,3,200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 200}",0.898051,0.895992,0.855764,0.882030,0.887436,0.883855,0.015193,1,0.925655,0.927107,0.931794,0.930156,0.929533,0.928849,0.002195
5,0.916454,0.085755,0.244512,0.007773,gbdt,2,200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 200}",0.890934,0.893559,0.854248,0.875350,0.883245,0.879467,0.014123,2,0.897088,0.895550,0.904111,0.899524,0.903050,0.899864,0.003305
7,0.883043,0.063561,0.259073,0.055040,gbdt,3,100,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 100}",0.889221,0.890147,0.850094,0.865103,0.873845,0.873682,0.015117,3,0.898610,0.898223,0.904874,0.899767,0.902137,0.900722,0.002484
4,0.913223,0.068902,0.309527,0.142400,gbdt,2,100,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 100}",0.886323,0.886275,0.844364,0.869951,0.868094,0.871001,0.015408,4,0.880646,0.878592,0.885503,0.884034,0.883697,0.882494,0.002511
17,1.282099,0.053001,0.205347,0.010365,dart,3,200,"{'lgbm__boosting_type': 'dart', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 200}",0.886470,0.887781,0.848408,0.864272,0.867280,0.870842,0.014767,5,0.889736,0.887357,0.896974,0.890117,0.889611,0.890759,0.003255
6,0.846487,0.024564,0.243666,0.002939,gbdt,3,50,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 50}",0.883241,0.883895,0.842496,0.859225,0.861366,0.866044,0.015731,6,0.880672,0.879041,0.888696,0.882015,0.882831,0.882651,0.003284
14,1.161232,0.049152,0.244548,0.031271,dart,2,200,"{'lgbm__boosting_type': 'dart', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 200}",0.876878,0.877726,0.836925,0.858777,0.858646,0.861790,0.014959,7,0.867883,0.867546,0.877151,0.872065,0.874380,0.871805,0.003710
16,0.993279,0.034106,0.236087,0.022731,dart,3,100,"{'lgbm__boosting_type': 'dart', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 100}",0.875530,0.876798,0.838683,0.852241,0.858422,0.860335,0.014421,8,0.869279,0.870268,0.879305,0.871736,0.875040,0.873126,0.003654
3,0.865320,0.052017,0.242376,0.006272,gbdt,2,50,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 50}",0.874252,0.872952,0.833351,0.854768,0.856747,0.858414,0.014876,9,0.863588,0.861220,0.871406,0.868172,0.869319,0.866741,0.003765
15,0.887081,0.049548,0.245447,0.004713,dart,3,50,"{'lgbm__boosting_type': 'dart', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 50}",0.872576,0.872024,0.834531,0.850582,0.856701,0.857283,0.014241,10,0.866124,0.864504,0.872777,0.869434,0.869320,0.868432,0.002879


С типом бустинга мы определились. Лучшие результаты показывает «gbdt». А вот другие параметры еще стоит поискать, ведь наилучшие качество модели достигнуто в крайних точках заданного нами пространства гиперпараметров.

In [50]:
param = {
    'lgbm__boosting_type': ['gbdt', ],
                            'lgbm__max_depth': [2, 3, 4],
'lgbm__n_estimators': [200, 400, 800]}
LGBM_grid = GridSearchCV(LGBM_pipe, param, scoring='roc_auc', cv=folds, return_train_score=True)
LGBM_grid.fit(x_train, y_train)
pd.DataFrame(LGBM_grid.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lgbm__boosting_type,param_lgbm__max_depth,param_lgbm__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
5,1.289411,0.168639,0.214285,0.028654,gbdt,3,800,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 800}",0.917047,0.912782,0.881172,0.897443,0.904727,0.902634,0.012670,1,0.981666,0.983217,0.986139,0.984264,0.985404,0.984138,0.001586
2,1.241905,0.039057,0.215105,0.008682,gbdt,2,800,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 800}",0.917102,0.912729,0.885696,0.894300,0.902699,0.902505,0.011554,2,0.950043,0.946992,0.952825,0.952387,0.950256,0.950501,0.002076
8,1.477884,0.084061,0.212129,0.008349,gbdt,4,800,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 4, 'lgbm__n_estimators': 800}",0.920055,0.900837,0.875285,0.896095,0.905863,0.899627,0.014578,3,0.996271,0.997539,0.997872,0.997408,0.997023,0.997223,0.000548
7,1.282039,0.043278,0.215540,0.004703,gbdt,4,400,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 4, 'lgbm__n_estimators': 400}",0.914790,0.905247,0.869339,0.894263,0.903622,0.897452,0.015492,4,0.979808,0.984147,0.985317,0.985281,0.983833,0.983677,0.002024
4,1.165407,0.014996,0.210741,0.004035,gbdt,3,400,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 400}",0.911553,0.905504,0.873348,0.892639,0.899823,0.896573,0.013188,5,0.956757,0.959987,0.961274,0.962150,0.962252,0.960484,0.002033
6,1.079175,0.064641,0.218450,0.023296,gbdt,4,200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 4, 'lgbm__n_estimators': 200}",0.905689,0.900482,0.862468,0.885611,0.898661,0.890582,0.015535,6,0.952861,0.959514,0.959365,0.960714,0.961066,0.958704,0.002995
1,1.161535,0.068825,0.219506,0.008041,gbdt,2,400,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 400}",0.903795,0.900558,0.865940,0.882099,0.891412,0.888761,0.013692,7,0.923693,0.920283,0.927149,0.923117,0.924969,0.923842,0.002255
3,0.971373,0.025596,0.250292,0.048966,gbdt,3,200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 3, 'lgbm__n_estimators': 200}",0.898051,0.895992,0.855764,0.882030,0.887436,0.883855,0.015193,8,0.925655,0.927107,0.931794,0.930156,0.929533,0.928849,0.002195
0,0.983637,0.098311,0.232470,0.009000,gbdt,2,200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 200}",0.890934,0.893559,0.854248,0.875350,0.883245,0.879467,0.014123,9,0.897088,0.895550,0.904111,0.899524,0.903050,0.899864,0.003305


Дальнейшее увеличение глубины деревьев не помогло. А вот увеличение их количества работает. Но растет и переобучение. Попробуем зафиксировать глубину на уровне 2, а число оценщиков увеличим еще больше.

In [51]:
param = {
    'lgbm__boosting_type': ['gbdt', ],
                            'lgbm__max_depth': [2, ],
'lgbm__n_estimators': [600, 800, 1000, 1200]}
LGBM_grid = GridSearchCV(LGBM_pipe, param, scoring='roc_auc', cv=folds, return_train_score=True)
LGBM_grid.fit(x_train, y_train)
pd.DataFrame(LGBM_grid.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lgbm__boosting_type,param_lgbm__max_depth,param_lgbm__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,1.448734,0.030511,0.205456,0.018289,gbdt,2,1200,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 1200}",0.920807,0.916331,0.893994,0.899363,0.909782,0.908055,0.010073,1,0.963023,0.961490,0.965915,0.964490,0.963986,0.963781,0.001478
2,1.317117,0.114685,0.210193,0.015895,gbdt,2,1000,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 1000}",0.918931,0.915994,0.890303,0.897370,0.906482,0.905816,0.010845,2,0.957570,0.955498,0.960488,0.959566,0.957892,0.958203,0.001726
1,1.270657,0.044847,0.236021,0.045552,gbdt,2,800,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 800}",0.917102,0.912729,0.885696,0.894300,0.902699,0.902505,0.011554,3,0.950043,0.946992,0.952825,0.952387,0.950256,0.950501,0.002076
0,1.162191,0.051175,0.226775,0.016953,gbdt,2,600,"{'lgbm__boosting_type': 'gbdt', 'lgbm__max_depth': 2, 'lgbm__n_estimators': 600}",0.909925,0.908391,0.875552,0.888837,0.898049,0.896151,0.012813,4,0.938840,0.935206,0.942485,0.941557,0.939572,0.939532,0.002531


Мы видим что повышение качества на тестовой выборке на одну тысячную ведет к повышению на тренировочной на 2-3 тысячные. Колебания качества модели на разных фолдах небольшое. Скорее всего, есть смысл просто выбрать наилучшую модель, если только модели других классов не превзойдут LGBM. К одной из них, логистической регрессии, перейдем прямо сейчас.

In [52]:
lr_pipe = Pipeline(
    [('preparation', preparation),
    ('ct', ct_ohe),
    ('poly', PolynomialFeatures()),
    ('lr', LogisticRegression(random_state=130323))])

In [53]:
param = {
    'poly__degree': [1, 2, 3],
                            'lr__penalty': ['l1', 'l2'],
'lr__C': [0.01, 0.1, 1],
'lr__solver': ['liblinear', 'sag']}
lr_grid = GridSearchCV(lr_pipe, param, scoring='roc_auc', cv=folds, return_train_score=True)
lr_grid.fit(x_train, y_train)
pd.DataFrame(lr_grid.cv_results_).sort_values(by='rank_test_score')

C:\Users\User\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\User\anaconda3\envs\practicum\l

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr__C,param_lr__penalty,param_lr__solver,param_poly__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
13,0.737931,0.139693,0.238417,0.013874,0.1,l1,liblinear,2,"{'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'poly__degree': 2}",0.857330,0.858908,0.814396,0.838052,0.834895,0.840716,0.016375,1,0.846728,0.847210,0.856364,0.851817,0.851648,0.850754,0.003526
24,0.905056,0.066181,0.203545,0.017017,1,l1,liblinear,1,"{'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'poly__degree': 1}",0.854115,0.858348,0.812155,0.838715,0.838706,0.840408,0.016210,2,0.842040,0.840724,0.852390,0.846149,0.844651,0.845191,0.004072
30,0.756881,0.056317,0.190836,0.029179,1,l2,liblinear,1,"{'lr__C': 1, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'poly__degree': 1}",0.853808,0.858234,0.812431,0.838936,0.838536,0.840389,0.016036,3,0.841991,0.840707,0.852289,0.846079,0.844626,0.845139,0.004045
33,0.971407,0.092580,0.181920,0.044278,1,l2,sag,1,"{'lr__C': 1, 'lr__penalty': 'l2', 'lr__solver': 'sag', 'poly__degree': 1}",0.853744,0.858124,0.812357,0.838812,0.838490,0.840305,0.016029,4,0.841977,0.840754,0.852279,0.846066,0.844632,0.845142,0.004033
14,1.490569,0.086624,0.271610,0.059394,0.1,l1,liblinear,3,"{'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'poly__degree': 3}",0.860632,0.856881,0.811336,0.836819,0.831927,0.839519,0.017924,5,0.853137,0.856875,0.864919,0.860621,0.859917,0.859094,0.003933
10,2.183827,0.538467,0.145701,0.053093,0.01,l2,sag,2,"{'lr__C': 0.01, 'lr__penalty': 'l2', 'lr__solver': 'sag', 'poly__degree': 2}",0.857165,0.854262,0.814230,0.838397,0.831963,0.839204,0.015660,6,0.849891,0.850492,0.859468,0.854032,0.854594,0.853695,0.003434
18,0.866096,0.104771,0.243191,0.036640,0.1,l2,liblinear,1,"{'lr__C': 0.1, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'poly__degree': 1}",0.852607,0.856023,0.812380,0.837818,0.836925,0.839151,0.015422,7,0.840293,0.839046,0.850342,0.844045,0.843135,0.843372,0.003931
21,0.719085,0.107549,0.174831,0.035343,0.1,l2,sag,1,"{'lr__C': 0.1, 'lr__penalty': 'l2', 'lr__solver': 'sag', 'poly__degree': 1}",0.852432,0.856074,0.811759,0.837891,0.837169,0.839065,0.015611,8,0.840296,0.839147,0.850327,0.844085,0.843134,0.843398,0.003905
12,0.691469,0.168828,0.163398,0.026092,0.1,l1,liblinear,1,"{'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'poly__degree': 1}",0.852941,0.855473,0.809729,0.835682,0.836589,0.838083,0.016341,9,0.838823,0.837368,0.849347,0.842623,0.841447,0.841922,0.004152
7,0.893747,0.115119,0.217786,0.020802,0.01,l2,liblinear,2,"{'lr__C': 0.01, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'poly__degree': 2}",0.856330,0.852538,0.812647,0.838453,0.827444,0.837482,0.016127,10,0.848268,0.848979,0.858576,0.852546,0.853341,0.852342,0.003683


Пожалуй, здесь нет перспективных направлений для подгонки гиперпараметров. Рассмотренные величины не так сильно влияют на качество модели, чтобы рассчитывать на приближение к LGBM. Возможно случайный лес проявит себя лучше.

In [54]:
rf_pipe = Pipeline(
    [('preparation', preparation),
    ('ct', ct_ohe),
    ('rf', RandomForestClassifier(random_state=130323))])

In [55]:
param = {
    'rf__n_estimators': [50, 100, 200, 500],
                            'rf__max_depth': list(range(2, 11, 2)),
'rf__min_samples_leaf': [1, 10, 25]}

rf_grid = GridSearchCV(rf_pipe, param, scoring='roc_auc', cv=folds, return_train_score=True)
rf_grid.fit(x_train, y_train)
pd.DataFrame(rf_grid.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__max_depth,param_rf__min_samples_leaf,param_rf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
38,1.558156,0.519509,0.196363,0.079002,8,1,200,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 200}",0.873794,0.868387,0.834189,0.847379,0.848960,0.854542,0.014554,1,0.920779,0.921027,0.927740,0.923669,0.923637,0.923370,0.002508
39,4.637965,0.338132,0.328479,0.128658,8,1,500,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 500}",0.873455,0.868804,0.833457,0.846337,0.849351,0.854281,0.014830,2,0.921120,0.921313,0.927650,0.923484,0.923636,0.923440,0.002352
37,1.291840,0.364935,0.211988,0.050943,8,1,100,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 100}",0.874599,0.867603,0.832177,0.846509,0.846640,0.853506,0.015457,3,0.919733,0.920356,0.927311,0.922886,0.923232,0.922704,0.002678
50,1.653344,0.005035,0.198779,0.004484,10,1,200,"{'rf__max_depth': 10, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 200}",0.874140,0.868671,0.832032,0.845694,0.846939,0.853496,0.015627,4,0.966337,0.968379,0.970640,0.968088,0.966732,0.968035,0.001516
51,4.058797,0.786189,0.337248,0.074538,10,1,500,"{'rf__max_depth': 10, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 500}",0.873698,0.867676,0.830868,0.846235,0.848817,0.853459,0.015465,5,0.967183,0.968263,0.970602,0.968015,0.967575,0.968328,0.001196
41,1.128054,0.209114,0.231189,0.063346,8,10,100,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 100}",0.871574,0.868089,0.832242,0.848394,0.845720,0.853204,0.014681,6,0.890985,0.892344,0.899324,0.896794,0.894875,0.894864,0.003001
42,1.772000,0.544922,0.276040,0.076363,8,10,200,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 200}",0.871015,0.866754,0.831731,0.848302,0.847529,0.853066,0.014269,7,0.890511,0.892216,0.899297,0.896817,0.895250,0.894818,0.003148
43,4.704186,0.377545,0.426140,0.115870,8,10,500,"{'rf__max_depth': 8, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 500}",0.871781,0.866296,0.831708,0.847818,0.847156,0.852952,0.014449,8,0.891130,0.892581,0.900186,0.896483,0.894991,0.895074,0.003159
55,4.736900,0.117993,0.460999,0.027207,10,10,500,"{'rf__max_depth': 10, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 500}",0.872111,0.866410,0.830725,0.846903,0.847386,0.852707,0.014896,9,0.904498,0.905022,0.912123,0.908834,0.907641,0.907623,0.002766
54,2.855685,0.235470,0.362303,0.041915,10,10,200,"{'rf__max_depth': 10, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 200}",0.872129,0.866415,0.830923,0.846387,0.847483,0.852667,0.014879,10,0.903862,0.905414,0.911991,0.908906,0.907620,0.907559,0.002819


Мы видим, что в районе значения roc_auc 0,855 находится предельное значение. Даже значительное увеличение числа деревьев приводит только к еще большему переобучению. Таким образом можем сделать вывод, что и случайный лес не сможет превзойти LGBM. Именно этому классификатору мы поручим предсказание на тестовой выборке.



In [56]:
LGBM_pipe.set_params(lgbm__boosting_type='gbdt',
    lgbm__max_depth=2,
    lgbm__n_estimators=1200)
LGBM_pipe.fit(x_train, y_train)
predictions = LGBM_pipe.predict(x_test)
prob = LGBM_pipe.predict_proba(x_test)[:, 1]
print('Значение roc_auc на тестовой выборке', roc_auc_score(y_test, prob))
pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

Значение roc_auc на тестовой выборке 0.9285286398432562


,0,1,accuracy,macro avg,weighted avg
precision,0.894126,0.840314,0.882453,0.867220,0.879856
recall,0.952859,0.687366,0.882453,0.820113,0.882453
f1-score,0.922559,0.756184,0.882453,0.839371,0.878438
support,1294.000000,467.000000,0.882453,1761.000000,1761.000000


In [57]:
lgbm_for_features = LGBMClassifier(boosting_type='gbdt', max_depth=2, n_estimators=1200, importance_type='gain')
lgbm_for_features.fit(new_x_train, y_train)
pd.Series(lgbm_for_features.feature_importances_,
    index=lgbm_for_features.feature_name_).sort_values(ascending=False)

long                                     10420.787140
Type_Two_year                             2013.891010
InternetService_Fiber_optic               1992.538861
TotalCharges                              1013.525409
MonthlyCharges                             797.733661
Type_One_year                              761.643349
PaymentMethod_Electronic_check             515.214193
InternetService_Nothing                    431.212127
PaperlessBilling_Yes                       223.846889
OnlineSecurity_Yes                         143.483489
StreamingMovies_Yes                        110.098834
StreamingTV_Yes                            100.252620
MultipleLines_Yes                           97.708828
SeniorCitizen                               87.812999
gender_Male                                 80.166112
Dependents_Yes                              63.518169
TechSupport_Yes                             58.371329
MultipleLines_Nothing                       43.720422
number_service              

*Выводы*
Что же, поставленная задача выполнена. Получена модель необходимого качества. Бонусом создана функция, объединяющая разрозненные данные и выделяющая целевой признак. Таким образом, в случае появления новой информации сделать прогноз можно будет за несколько строк.
Обратим внимание на уровень других метрик. Модели уже сейчас удается находить более 68 % клиентов, готовых разорвать договор. С вероятностью 88 % компания может определить дальнейшие действия клиентов. При этом есть потенциал улучшения этих показателей, ведь они чувствительны к дисбалансу классов, который в этом проекте мы не стали компенсировать.
Также выше показан рейтинг важности признаков с точки зрения модели. Наибольший вес, причем с огромным отрывом, имеет продолжительность действия текущего договора. Среди прочих параметров, как и ожидалось, значим тип договора, тип интернет соединения, ежемесячная плата и метод оплаты. Возможно в этих параметрах есть недоработки, исправление которых снизит отток клиентов.


## Отчет
Целью проекта было построение модели, предсказывающей отказ клиента от услуг связи. Для этого компания предоставила данные о пользователях, наборе выбранных ими услуг и условиях договора.

При первом рассмотрении данных выяснилось, что у нас есть 4 таблицы. У них был один общий столбец, на основе которого их можно объединить. Также важной особенностью стало большое количество категориальных признаков. Кроме них было два числовых показателя и две даты. К тому же тип данных для трех из четырех некатегориальных столбцов был определен неверно. Как таковой целевой признак в таблицах отсутствовал, но быстро выявился столбец с датами расторжения договоров, на основе которого его можно сконструировать.
В остальном данные были подготовлены: отсутствовали пропуски, дубликаты. Кроме нескольких странных значений в одном из числовых столбцов придраться не к чему.

На этапе разведочного анализа, пользуясь наблюдениями предварительного исследования данных, объединили все таблицы, создали целевой показатель. Это привело к возникновению пропущенных данных.
При заполнении пропусков почти во всех столбцах использовали слово «No». Это соответствует смыслу данных в столбцах и не создает лишние категории. Исключение было сделано для двух столбцов, где это слово могло привести к путанице. 
В данных присутствовали значения, препятствующие дальнейшему анализу. В столбце с общими платежами за все время действия контракта встречалось значение « ». Оно связано с тем, что для ряда клиентов не прошло ни одного платежа. Соответственно пробелы были заменены на нули.
В столбце с датами завершения действия договоров было много значений «No». Вместо них поставили дату выгрузки данных. Это позволило преобразовать данные в правильный формат и использовать их для конструирования новых признаков.

С этого мы и начали этап разведочного анализа. Были сформированы 3 признака: продолжительность действия договора, количество оказываемых услуг и категориальный признак, указывающий тип оказываемых клиенту услуг: интернет, телефон или и то, и другое.
Затем каждый признак был рассмотрен для двух групп: ушедшие и действующие клиенты. Получилось, что ушедший клиент, скорее всего, пользуется услугами компании 1-2 года, оплачивает услуги раз в месяц с помощью электронных чеков, не имеет детей и подключен к интернету через оптико-волоконный кабель. В целом удалось найти несколько потенциально проблемных точек во взаимоотношениях с клиентами, работа над которыми может снизить отток.
Далее была рассмотрена корреляция между признаками. Так как многие из них носят категориальный характер, вместо обычной корреляции использовались сходные по своему предназначению коэффициенты, рассчитанные с помощью библиотеки phik. Ряд показателей продемонстрировали значительную связь. В дальнейшем на основе этого наблюдения из данных были удалены признаки, относящиеся к датам. Некоторые признаки, несмотря на значительную корреляцию, были оставлены, так как эксперименты показали, что они положительно влияют на качество моделей, а интерпретируемость не изменяется.

Перед построением первых моделей были созданы две вспомогательные функции. Первая повторяла наши действия по созданию целевого показателя и объединению данных. В дальнейшем она позволит быстро применять новые данные, если такие будут предоставлены. Вторая предназначена для автоматизации подготовки данных, проведенной на предыдущих этапах. Она стала частью pipeline.
На примере случайного леса и логистической регрессии с регуляризацией рассмотрена значимость признаков для предсказания результата. Оба показали высокое значение продолжительности действия контракта, суммарных платежей по нему, размера ежемесячных выплат, типа Интернет-соединения и типа договора. В отношении многих других признаков мнения моделей разошлись, но это не так важно, ведь их значимость по сравнению с перечисленными невелика.
Далее мы испытали ряд моделей без подбора гиперпараметров. Их результаты приведены в таблице ниже.


In [58]:
pd.DataFrame(result, index=['valid', 'train']).T.sort_values(by='valid', ascending=False)

,valid,train
LGBMClassifier,0.887078,0.983759
LogisticRegression,0.840302,0.845142
random_forest,0.837934,0.999994
RidgeClassifier,0.831016,0.835334
ExtraTreesClassifier,0.804361,0.999999
KNeighborsClassifier,0.774999,0.893899
SVC,0.761127,0.943571
DummyClassifier,0.500000,0.500000


Три лучшие модели из списка были признаны наиболее перспективными. На заключительном этапе исследования для них подбирались гиперпараметры, позволяющие улучшить качество и, при необходимости, снизить переобученность. В таблице ниже приведены наилучшие полученные параметры для каждой из них.

In [59]:
pd.DataFrame([['boosting_type - gbdt, max_depth - 2, n_estimators - 1200', 0.93],
         ['max_depth - 8, min_samples_leaf - 1, n_estimators - 200', 0.85],
         ['C - 0.1, penalty - l1, solver - liblinear, poly__degree - 2', 0.84]],
         index=['LGBMClassifier', 'RandomForestClassifier', 'LogisticRegression'],
         columns=['params', 'roc_auc'])

,params,roc_auc
LGBMClassifier,"boosting_type - gbdt, max_depth - 2, n_estimators - 1200",0.93
RandomForestClassifier,"max_depth - 8, min_samples_leaf - 1, n_estimators - 200",0.85
LogisticRegression,"C - 0.1, penalty - l1, solver - liblinear, poly__degree - 2",0.84


На тестовой выборке проверена модель LGBMClassifier. Причем на ней результат оказался даже лучше, чем при кросс-валидации. Видимо, немного дополнительных данных помогло.

In [60]:
print('Значение roc_auc на тестовой выборке', roc_auc_score(y_test, prob))
pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

Значение roc_auc на тестовой выборке 0.9285286398432562


,0,1,accuracy,macro avg,weighted avg
precision,0.894126,0.840314,0.882453,0.867220,0.879856
recall,0.952859,0.687366,0.882453,0.820113,0.882453
f1-score,0.922559,0.756184,0.882453,0.839371,0.878438
support,1294.000000,467.000000,0.882453,1761.000000,1761.000000


На финальной модели посмотрели важность различных признаков. В целом, список совпал с тем, который был получен ранее. С грустью отметим, что два из трех искусственно сконструированных признака оказались бесполезными. Зато третий – продолжительность действия контракта, с отрывом самый важный.

In [61]:
pd.Series(lgbm_for_features.feature_importances_,
    index=lgbm_for_features.feature_name_).sort_values(ascending=False)

long                                     10420.787140
Type_Two_year                             2013.891010
InternetService_Fiber_optic               1992.538861
TotalCharges                              1013.525409
MonthlyCharges                             797.733661
Type_One_year                              761.643349
PaymentMethod_Electronic_check             515.214193
InternetService_Nothing                    431.212127
PaperlessBilling_Yes                       223.846889
OnlineSecurity_Yes                         143.483489
StreamingMovies_Yes                        110.098834
StreamingTV_Yes                            100.252620
MultipleLines_Yes                           97.708828
SeniorCitizen                               87.812999
gender_Male                                 80.166112
Dependents_Yes                              63.518169
TechSupport_Yes                             58.371329
MultipleLines_Nothing                       43.720422
number_service              

*Выводы*
Поставленная цель создание модели определенного качества была достигнута. Предварительно сформированный план выполнен.
Основной сложностью при выполнении проекта стала работа с большим числом категориальных признаков при почти полном отсутствии числовых. С технической точки зрения наибольшей сложностью стал перенос максимального числа действий по подготовке данных в конвейер.
Глядя на конвейер можно выделить основные этапы работы модели: предварительная подготовка данных и создание дополнительных признаков, кодирование для категориальных столбцов и нормализация для числовых, подбор модели с наилучшими гиперпараметрами.
После ряда экспериментов лучшей была признана модель LGBMClassifier с 1200 оценщиками, глубиной деревьев 2 и типом бустинга «gbdt». Она позволила достичь на тестовой выборке roc_auc около 0,93. Неплохими оказались и другие метрики качества модели.
